In [ ]:
import json
import time
import sys
from sqlalchemy import func, select, sql, update
import logbook
import database
from database.schema import revision_table, Rating

In [ ]:
importance_file = "archive/04b_find_importance/2017-12-15 19:57:25 81807fc/importance.utf8.tsv"
#importance_file = "archive/04b_find_importance/2017-10-03 13:54:30 14665f3/importance.utf8.tsv"
project_file = "data/projects-2016-10-14-dedup.json"
exp_name = "05c_update_importance"
exp = logbook.Experiment(exp_name)
log = exp.get_logger()

In [ ]:
project_ids = []
projects_by_name = {}
with open(project_file, "rb") as f:
    # Skip header
    f.next()
    for row in f:
        data = json.loads(row)
        project_ids.append(data["project_id"])
        projects_by_name[data["project_name"]] = data

In [ ]:
project_important = dict((project_id, set()) for project_id in project_ids)
skipped_projects = set()
with open(importance_file, "rb") as f:
    for row in f:
        row = row.decode('utf-8')
        title, unique, page_id, importance = row.strip().split(u'\t')
        if importance == 'Top' or importance == 'High':
            try:
                project_id = projects_by_name[unique]["project_id"]
                project_important[project_id].add(int(page_id))
            except KeyError:
                skipped_projects.add(unique)

In [ ]:
conn = database.engine.connect()
try:
    for project_id in project_ids:
        log.info("Starting project: %d" % project_id)
        tab = revision_table(project_id).__table__
        for page_id in project_important[project_id]:
            stmt = tab.update().where(tab.c.article_id == page_id) \
                .values(important=True)
            conn.execute(stmt)
finally:
    conn.close()

In [ ]:
conn = database.engine.connect()
try:
    for project_id in project_ids:
        log.info("Starting project: %d" % project_id)
        tab = Rating.__table__
        for page_id in project_important[project_id]:
            stmt = tab.update().where(
                sql.and_(
                    tab.c.project_id == project_id,
                    tab.c.article_id == page_id)) \
                .values(important=True)
            conn.execute(stmt)
finally:
    conn.close()
log.info("Script complete")